# Loan pre-payment analysis

Create Date: 08/June/2020
Last updated: 

Home Loan repayment analysis before even starting the EMI


### re-payment schedule

1. private firm till 60 age loan re-payment.
2. Gov job till age 70 repayment possible.


### Loan eligibility check on 08/jun/2020

SBI is ready to give upto 80%.


https://cleartax.in/s/home-loan-tax-benefit

In [1]:
# Basic constants involved.
sqft = 1600
price_sqft = 5100
car_charge = 350000

# Total price of an asset without any tax or other charges.
total_asset_value = sqft * price_sqft + car_charge

# 1% cess extra
gst_f = lambda x: (x / 100) * (7.5 + 1)

land_value_f = lambda total_v: total_v * (1/3)
const_cost_f = lambda total_v: total_v * (2/3)
reg_f = lambda land_v: (land_v / 100 ) * 7.0

gst = gst_f(const_cost_f(total_asset_value))
reg_charge = reg_f(land_value_f(total_asset_value))

In [22]:
land_value_f(total_asset_value)

2836666.6666666665

In [143]:
const_cost_f(total_asset_value)

5673333.333333333

In [158]:
print(f"Gst for construction: {gst}")

Gst for construction: 482233.3333333333


In [34]:
grant_total = total_asset_value + gst + reg_charge + 200000
grant_total

9390800.0

In [20]:
loan_amount = total_asset_value + gst

In [25]:
# loan_amount * (8/10)
loan_amount

8992233.333333334

In [4]:
co_pay = loan_amount / 4
exp_loan_amount = loan_amount * ( 3 /4)

In [19]:
exp_loan_amount

6744175.0

In [5]:
# Initial allocation of 36%
# 
alloc1 = loan_amount * (36 / 100)
copay1 = alloc1 / 4
loan1 = alloc1 * (3/4)

In [231]:
copay1, loan1, exp_loan_amount

(809301.0, 2427903.0, 6744175.0)

## Co-pay option for on-going projects

In [295]:
def pre_emi_calculation(exp_loan_amount,
          allocation,
          block_cont_ratio=0.25,
          interest=8.0, init_full=False, pay_int=True, special=False, special_amount=1000000):
    """Quarter wise interest calculation with co-payment option.
    
    Only interest will be calculated till the building is ready to occupy.
    
    @param exp_loan_amount: 
    
    """
    print(f"Allocation: {allocation}")
    
    init_alp = allocation[0]
    # Special schema based on custom division.
    if special:
        init_block = exp_loan_amount * (init_alp[0] / 100)
        init_self = special_amount
        init_loan = init_block - init_self
        base_int = (init_loan * (interest / 100)) / 12
    else:
        init_block = exp_loan_amount * (init_alp[0] / 100)
        init_loan = init_block * (1 - block_cont_ratio)
        init_self = init_block * (block_cont_ratio)
        base_int = (init_loan * (interest / 100)) / 12
    
    print (init_block)
    
    if init_full:
        int_total = 0
        curr_loan = 0
        curr_self = init_block
        base_int = 0
    else:
        int_total = base_int
        curr_loan = init_loan
        curr_self = init_self
        
    if not pay_int:
        curr_loan += base_int
        base_int = 0
        int_total = 0
        
        
    print(f"Initial Loan borrow Interest: {base_int}")
    print(f"Initial Loan: {curr_loan}, Initial Invest: {curr_self} ")
    
    block_interest = []
    block_interest.append(base_int * init_alp[1])
    
    allocs = allocation[1:]
    
    for alloc in allocs:
        next_blok = exp_loan_amount * (alloc[0] / 100)
        # 75% vs 25%
        next_loan_part = next_blok * (1 - block_cont_ratio)
        next_self_part = next_blok * block_cont_ratio
        curr_self += next_self_part
        curr_loan += next_loan_part
        
        base_int = (curr_loan * (interest / 100)) / 12
        
        if not pay_int:
            curr_loan += base_int
            base_int = 0
        
        # Monthly payments
        # 1. interest for already dispersed 
        # 2. Accumulate for next installment.
        print(f"Curr Loan: {curr_loan: .2f}, Curr Self: {curr_self : .2f} QLBlock: {next_loan_part :.2f}, Qself({block_cont_ratio :.2f}): {next_self_part : .2f}, QInt: {base_int *  alloc[1] : .2f}, MPay: {base_int : .2f}")
        int_total += base_int * alloc[1]
        block_interest.append(base_int * alloc[1])
    
    self_total = exp_loan_amount - curr_loan
    self_gtotal = self_total + int_total
    print(f"\nLoan: {exp_loan_amount}, Total pre-emi interest: {int_total}, principle: {curr_loan}, self: {self_total}, self Priciple + Interest: {self_gtotal}")
    return block_interest
    

In [291]:
# 1000000 - (loan_amount * (16 / 100))
# loan_amount
# allocation = [36, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 3, 2, 1, 1]
# special_allocation = [16, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
new_allocation = [16, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 8, 1, 1]
# sum(new_allocation), sum(special_allocation), sum(allocation)

# Initial quoat
alloc_with_month_org = [(36, 1), (6, 3), (6, 3), (6, 3),
                    (6, 3), (6, 3), (6, 3),
                    (6, 2), (6, 2), (6, 2), (3, 4), (3, 3),
                    (2, 6), (1, 6),
                    (1, 1)
                   ]

# Allocation with the duration of month for the next allocation.
alloc_with_month = [(16, 1), (6, 3), (6, 3), (6, 3),
                    (6, 3), (6, 3), (6, 3),
                    (6, 2), (6, 2), (6, 2), (6, 2), (6, 2),
                    (8, 2), (8, 7),
                    (1, 7), (1, 1)
                   ]
# print(sum([i[0] for i in alloc_with_month]))
loan_amount

8992233.333333334

In [282]:
# sum([i[0] for i in alloc_with_month_org])

## Interest impact using standard allocation

In [312]:
int_blocks = pre_emi_calculation(
    exp_loan_amount=loan_amount,
    allocation=alloc_with_month,
    interest=7.65,
    pay_int=True,
    block_cont_ratio=0.35, special=True, special_amount=1000000)

Allocation: [(16, 1), (6, 3), (6, 3), (6, 3), (6, 3), (6, 3), (6, 3), (6, 2), (6, 2), (6, 2), (6, 2), (6, 2), (8, 2), (8, 7), (1, 7), (1, 1)]
1438757.3333333335
Initial Loan borrow Interest: 2797.078000000001
Initial Loan: 438757.3333333335, Initial Invest: 1000000 
Curr Loan:  789454.43, Curr Self:  1188836.90 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  15098.32, MPay:  5032.77
Curr Loan:  1140151.53, Curr Self:  1377673.80 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  21805.40, MPay:  7268.47
Curr Loan:  1490848.63, Curr Self:  1566510.70 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  28512.48, MPay:  9504.16
Curr Loan:  1841545.73, Curr Self:  1755347.60 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  35219.56, MPay:  11739.85
Curr Loan:  2192242.83, Curr Self:  1944184.50 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  41926.64, MPay:  13975.55
Curr Loan:  2542939.93, Curr Self:  2133021.40 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  48633.73, MPa

In [314]:
int_blocks = pre_emi_calculation(
    exp_loan_amount=loan_amount,
    allocation=alloc_with_month_org,
    interest=7.65,
    pay_int=True,
    block_cont_ratio=0.35, special=True, special_amount=1000000)

Allocation: [(36, 1), (6, 3), (6, 3), (6, 3), (6, 3), (6, 3), (6, 3), (6, 2), (6, 2), (6, 2), (3, 4), (3, 3), (2, 6), (1, 6), (1, 1)]
3237204.0
Initial Loan borrow Interest: 14262.1755
Initial Loan: 2237204.0, Initial Invest: 1000000 
Curr Loan:  2587901.10, Curr Self:  1188836.90 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  49493.61, MPay:  16497.87
Curr Loan:  2938598.20, Curr Self:  1377673.80 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  56200.69, MPay:  18733.56
Curr Loan:  3289295.30, Curr Self:  1566510.70 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  62907.77, MPay:  20969.26
Curr Loan:  3639992.40, Curr Self:  1755347.60 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  69614.85, MPay:  23204.95
Curr Loan:  3990689.50, Curr Self:  1944184.50 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  76321.94, MPay:  25440.65
Curr Loan:  4341386.60, Curr Self:  2133021.40 QLBlock: 350697.10, Qself(0.35):  188836.90, QInt:  83029.02, MPay:  27676.34
Curr Loan:  469

In [267]:
sum([i[0] for i in zip(int_blocks, [i[1] for i in alloc_with_month])])

1394614.4599000001

In [309]:
ipm = lambda pr, interest: ((pr * interest) / 100) / 12
maxgain_ipm = lambda pr, interest, deposit: ipm(pr - deposit, interest)

ipm(5671565.794072523, 8.0)

37810.43862715015

In [311]:
maxgain_ipm(5671565.794072523, 8.0, 200000)

36477.10529381682

# Simple interest vs compounding interest

In [372]:
def calculate_si_emi(principle, emi, tensure):
    pass


def calculate_ci_emi(principle, interest, tenure):
    """
    EMI = P × r × (1 + r)n/((1 + r)n - 1) 
    where P= Loan amount, r= interest rate, n=tenure in number of months.
    
    @param principle: Total loan amount.
    @param interest: In percentage format.
    @param tenure: Year.
    """
    p = principle
    
    # Interest per month.
    r = interest / 100 / 12
    n = tenure * 12
    
    den = (1 + r) ** n
    #print (r, den, den/(den-1))
    
    return p * r * den / (den - 1)


def maxigain_interest():
    pass


def calculate_emi(principle, interest=7.65, simple=False, tenure=15):
    """
    For a given principle calculate the interest, compound vs simple.
    """
    pass

In [376]:
calculate_ci_emi(5000000, 7.65, 15)

46777.839406314546

In [357]:
5000000 * 0.765 * 1.000001727724937

3825006.6085478836

In [360]:
1.765 ** 60 / (1.765 ** 60 - 1)

1.0000000000000016

In [364]:
(100000 * (10/1200) * (1+(10/1200))**240) / ((1+(10/1200))**240 - 1)

965.0216450740089

# TODO

1. Compare the flexi-pay interest scheme and normal
2. EMI and number of years.
3. Simple interest calculation.

1. If project extends by an month will cause per month interest on the release amount accumulates.
2. If loan plans aren't great, we may stop as going with enough liquid cash helps with the interest handling. May be close to 40% of the total value.
3. Backout if the loan plans aren't that great.